In [1]:
url = 'https://anaconda.org/conda-forge/libta-lib/0.4.0/download/linux-64/libta-lib-0.4.0-h516909a_0.tar.bz2'
!curl -L $url | tar xj -C /usr/lib/x86_64-linux-gnu/ lib --strip-components=1
url = 'https://anaconda.org/conda-forge/ta-lib/0.4.19/download/linux-64/ta-lib-0.4.19-py37ha21ca33_2.tar.bz2'
!curl -L $url | tar xj -C /usr/local/lib/python3.7/dist-packages/ lib/python3.7/site-packages/talib --strip-components=3

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3745    0  3745    0     0  17022      0 --:--:-- --:--:-- --:--:-- 17022
100  503k  100  503k    0     0   798k      0 --:--:-- --:--:-- --:--:-- 4767k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3753    0  3753    0     0  17786      0 --:--:-- --:--:-- --:--:-- 17786
100  406k  100  406k    0     0   715k      0 --:--:-- --:--:-- --:--:--  715k


In [2]:
!pip install python-binance
!pip install binance-client

In [2]:
import pandas as pd
import numpy as np

In [3]:
from get_data import crypto_data

In [4]:
df = crypto_data()

## Prepare dataset

In [5]:
example = df["df_BTC"]
df_input = pd.DataFrame(columns = example.columns)
for single in df.values():
  df_input = df_input.append(single)

In [6]:
df_output = np.zeros(df_input.shape[0])
for i in range(1, df_input.price.shape[0]):
  diff = df_input.price.iloc[i] - df_input.price.iloc[i-1]
  if diff > 0:
    df_output[i] = 1
  elif diff < 0:
    df_output[i] = -1

In [7]:
df_input_test = df_input.iloc[::5]
df_input_train = df_input.iloc[1::5]
df_input_train = df_input_train.append(df_input.iloc[2::5])
df_input_train = df_input_train.append(df_input.iloc[3::5])
df_input_train = df_input_train.append(df_input.iloc[4::5])

In [8]:
df_output_test = df_output[::5]
df_output_train = df_output[1::5]
df_output_train = np.concatenate((df_output_train, df_output[2::5]))
df_output_train = np.concatenate((df_output_train, df_output[3::5]))
df_output_train = np.concatenate((df_output_train, df_output[4::5]))

## Decision trees

In [9]:
from sklearn import tree
clf = tree.DecisionTreeClassifier(max_depth=7)
clf = clf.fit(df_input_train, df_output_train)
print("decision tree score test: ", clf.score(df_input_test, df_output_test))
print("decision tree score train: ", clf.score(df_input_train, df_output_train))

decision tree score test:  0.632009233432721
decision tree score train:  0.6370566309967537


## Naive bayes

In [10]:
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB

In [11]:
gnb = GaussianNB()
gnb.fit(df_input_train, df_output_train)
print("Gaussian NB score test: ", gnb.score(df_input_test, df_output_test))
print("Gaussian NB score train: ",gnb.score(df_input_train, df_output_train))

gnb = MultinomialNB()
gnb.fit(df_input_train, df_output_train)
print("Multinomial NB score test: ", gnb.score(df_input_test, df_output_test))
print("Multinomial NB score train: ", gnb.score(df_input_train, df_output_train))

Gaussian NB score test:  0.18659228623641436
Gaussian NB score train:  0.18929902609113863
Multinomial NB score test:  0.25478503414446474
Multinomial NB score train:  0.25343272814716844


## Bayesian network

# TODO

## Random Forest

In [12]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=200, max_depth=20, random_state=0)
clf.fit(df_input_train, df_output_train)
print("random forest score test: ", clf.score(df_input_test, df_output_test))
print("random forest score train: ", clf.score(df_input_train, df_output_train))

random forest score test:  0.6279696066172934
random forest score train:  0.9431285319225682


## Boosting random forest with weghitng with evolutionara algorithm

# TODO

## SUPPORT VECTOR MACHINES

ktoś chce to sie może pobawić paramkami

In [13]:
from sklearn import svm
#for gamma in []
clf = svm.SVC()
clf.fit(df_input_train, df_output_train)
print("random forest score test: ", clf.score(df_input_test, df_output_test))
print("random forest score train: ", clf.score(df_input_train, df_output_train))

random forest score test:  0.4704241608156199
random forest score train:  0.4614404232295299


## Neural network

### Classifier same as in previous example

In [17]:
import tensorflow as tf
from tensorflow import keras
from ten

In [19]:
df_output

array([0., 1., 1., ..., 1., 1., 0.])

In [20]:
#model preparation
ds_output_NN_train = tf.one_hot(df_output_train, depth = 2)
ds_output_NN_test = tf.one_hot(df_output_test, depth = 2)

In [28]:
df_input.shape[1]

26

In [53]:
model = tf.keras.models.Sequential([                    
  #tf.keras.layers.Conv1(filters= 2, kernel_size = 3, activation='tanh', use_bias=False),
  #tf.keras.layers.MaxPool2D(),
  #tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(10, use_bias=True, activation="relu", input_shape=(26,)),
  tf.keras.layers.Dense(5, use_bias=True, activation="tanh"),
  tf.keras.layers.Dense(2, use_bias=True, activation="sigmoid")
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.0008),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.BinaryAccuracy()],
)

model.fit(
    df_input_train,
    ds_output_NN_train,
    epochs=4,validation_data = (df_input_test, ds_output_NN_test)
)

Epoch 1/4


/usr/local/lib/python3.7/dist-packages/keras/backend.py:4994: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`binary_crossentropy` received `from_logits=True`, but the `output`'


1287/1300 [============================>.] - ETA: 0s - loss: 0.5676 - binary_accuracy: 0.6887

/usr/local/lib/python3.7/dist-packages/keras/backend.py:4994: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`binary_crossentropy` received `from_logits=True`, but the `output`'


1300/1300 [==============================] - 4s 2ms/step - loss: 0.5672 - binary_accuracy: 0.6891 - val_loss: 0.5107 - val_binary_accuracy: 0.7357
Epoch 2/4
1300/1300 [==============================] - 3s 2ms/step - loss: 0.5141 - binary_accuracy: 0.7304 - val_loss: 0.5098 - val_binary_accuracy: 0.7349
Epoch 3/4
1300/1300 [==============================] - 3s 2ms/step - loss: 0.5141 - binary_accuracy: 0.7303 - val_loss: 0.5097 - val_binary_accuracy: 0.7349
Epoch 4/4
1300/1300 [==============================] - 3s 2ms/step - loss: 0.5143 - binary_accuracy: 0.7302 - val_loss: 0.5102 - val_binary_accuracy: 0.7346
